In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import AutomationModule as AM
from scipy.optimize import curve_fit
from natsort import natsorted




In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a=5/3

In [ ]:
slices = [0.1,0.15,0.3,0.5,0.7,0.9,1.0]
#Calculate T(L) for given frequencies 
frequencies_check = np.array([0.418,0.462,0.471])
transmission_total_l= []
for path_direction in [
                       "RCP Sample 1 Slices SAL Gap Formation/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                transmission0 = sim_data0['flux1'].flux
                transmission = sim_data['flux1'].flux
                transmission_normalized = transmission / transmission0
                monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
                reduced_freqs = a/monitor_lambdas
                indices = [np.argmin(np.abs(reduced_freqs-item)) for item in frequencies_check]

                transmission_total_l +=  [transmission_normalized[indices]]

            


In [ ]:
def Td(L,T0,xi,l:float=1.12):
    Db = 1+2*3.25/xi
    Td = (1+3.25)*(l/Db)/xi *np.exp(-L/xi)
    return T0*Td

def Tb(L,l):
    return np.exp(-L/l) 

In [ ]:
frequencies_labels = np.array(["SAL","Near gap","PGB"])
l = [1.12,0.76,0.72]
xi = [8,3.6,5.1]
T0 = [1.,0.22,0.055]
z0 = 3.25


for j, nu in enumerate(frequencies_check):
    plt.figure(figsize=(15, 10))
    Ls = np.linspace(0.1,18.01,100)
    plt.plot(
             np.log10(np.array(slices)*18.01),
             np.log10(np.array(transmission_total_l)[:,j]), 
             "o")
    
    plt.plot(
             np.log10(Ls),
             np.log10(np.exp(-Ls)/(l[j])),
             '--',
             color="red")

    #popt, pcov = curve_fit(lambda L, T0,xi: Td(L,T0,xi,l=l[j]),(np.array(slices)*18.01)[3:], (np.array(transmission_total_l)[:,j])[3:])

    plt.plot(
        np.log10(Ls),
        np.log10(Td(Ls,T0[j],xi[j],l[j])),
                 '--',color="green")
    
    plt.title(f'$\\nu$={nu} - {frequencies_labels[j]}')
    plt.ylabel("T(L)")
    plt.xlabel("L/a")
    plt.xlim(-1,1.5)
    plt.ylim(-4,0)
    plt.xticks([-1,0,1,1.5])
    plt.yticks([-4,-3,-2,-1,0])
    plt.show()

In [ ]:
slices = [0.1,0.15,0.3,0.5,0.7,0.9,1.0]

for path_direction in [
                       "RCP Sample 1 Slices SAL Gap Formation/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                plt.figure(figsize=(15, 10))
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                transmission0 = sim_data0['flux1'].flux
                transmission = sim_data['flux1'].flux
                transmission_normalized = transmission / transmission0
                transmission_flux_right = sim_data['flux1'].flux
                transmission_flux_left = sim_data['flux2'].flux
                monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
                plt.scatter(a/monitor_lambdas, np.log10(transmission_flux_left/transmission0), s=0.8, label='Left Monitor')
                plt.scatter(a/monitor_lambdas, np.log10(transmission_flux_right/transmission0), s=0.8, label='Right Monitor')
                plt.xlabel(r'$\nu$')
                plt.ylabel('T')
                plt.legend()
                plt.grid()
                plt.title(f"L={slices[i]}")
                plt.savefig(f'output/anderson/Convergence test L={slices[i]}.pdf', format='pdf')
                plt.show()

               



In [ ]:
slices = [0.1,0.15,0.3,0.5,0.7,0.9,1.0]

plt.figure(figsize=(15, 10))
for path_direction in [
                       "RCP Sample 1 Slices SAL Gap Formation/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                transmission0 = sim_data0['flux1'].flux
                transmission = sim_data['flux1'].flux
                transmission_normalized = transmission / transmission0
                transmission_flux_right = sim_data['flux1'].flux
                transmission_flux_left = sim_data['flux2'].flux
                monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
                plt.plot(a/monitor_lambdas, np.log10(transmission_flux_right/transmission0), label=f"L={slices[i]}")


plt.xlabel(r'$\nu$')
plt.ylabel('T')
plt.legend()
plt.grid()
plt.title("Bandgap formation")
plt.savefig(f'output/anderson/Bandgap formation.pdf', format='pdf')
plt.show()

               



In [ ]:
slices = [0.1,0.15,0.3,0.5,0.7,0.9,1.0]



def line_fit(x,m,b):
    return m*x+b


plt.figure(figsize=(15, 10))

for path_direction in [
                       "RCP Sample 1 Slices SAL Gap Formation/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                a = 5/3

                flux_t = sim_data["time_monitorT"].flux
                flux_index = np.argmax(flux_t.values)
                x,y = flux_t['t'][flux_index:]*1e12,np.log(flux_t[flux_index:])
                # Remove NaN and inf values
                mask = np.isfinite(y)
                x_clean = x[mask]
                y_clean = y[mask]

                popt, pcov = curve_fit(line_fit, x_clean, y_clean)
                plt.scatter(x_clean,y_clean,s=0.5,label=f"L={slices[i]:.3g}L - slope:{popt[0]:.3g}")
               


range = f'lambda_range: {a/structure_1.monitor_lambdas[0]:.3g} - {a/structure_1.monitor_lambdas[-1]:.3g}'

plt.ylabel("Log(Flux)")
plt.xlabel("Time[ps]")
plt.title("Range: "+range)
plt.legend()
plt.grid()
plt.savefig(f'output/anderson/Flux Time 20ps.pdf', format='pdf')
plt.show()



In [ ]:
slices = [0.1,0.15,0.3,0.5,0.7,0.9,1.0]
plt.figure(figsize=(15, 10))

for path_direction in [
                       "RCP Sample 1 Slices SAL Time Transmission Near Gap/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                a = 5/3

                flux_t = sim_data["time_monitorT"].flux

                plt.scatter(flux_t['t'][:]*1e12,flux_t[:],s=0.5,label=f"L={slices[i]}L")
               


range = f'lambda_range: {a/structure_1.monitor_lambdas[-1]:.3g} - {a/structure_1.monitor_lambdas[0]:.3g}'

plt.ylabel("Flux")
plt.xlabel("Time[ps]")
plt.title("Range: "+range)
plt.legend()
plt.grid()
plt.savefig(f'output/anderson/Flux Time Near Gap.pdf', format='pdf')
plt.show()



In [ ]:
slices =np.array([0.1,0.15,0.3,0.5,0.7,0.9,1.0])

plt.figure(figsize=(15, 10))

def line_fit(x,m,b):
    return m*x+b

max_times = []




for path_direction in [
                       "RCP Sample 1 Slices SAL Time Transmission Near Gap/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                flux_t = sim_data["time_monitorT"].flux
                flux_index = np.argmax(flux_t.values)
                x,y = flux_t['t'][flux_index:]*1e12,np.log((flux_t[flux_index:]/np.max(flux_t[flux_index:])))
                # Remove NaN and inf values
                mask = np.isfinite(y)
                x_clean = x[mask]
                y_clean = y[mask]

                plt.plot(x_clean,y_clean,"o",label=f"L={slices[i]:.3g}")
                max_times +=[flux_t['t'][flux_index]*1e12]
                #popt, pcov = curve_fit(line_fit, x_clean, y_clean)
                #plt.plot(x_clean,y_clean,"o",label=f"L={slices[i]:.3g}L - slope:{popt[0]:.3g}")
               


range = f'lambda_range: {a/structure_1.monitor_lambdas[0]:.3g} - {a/structure_1.monitor_lambdas[-1]:.3g}'

plt.ylabel("Log(Flux)")
plt.xlabel("Time[ps]")
plt.title("Range: "+range)
plt.legend()
plt.grid()
plt.savefig(f'output/anderson/Flux Time log Near Gap.pdf', format='pdf')
plt.show()




In [ ]:
print(max_times)
plt.figure(figsize=(15, 10))
plt.plot(slices*18.01,max_times,"o")
plt.ylabel(r"$t_{max}[ps]$")
plt.xlabel(r"L/a")
plt.show()

In [ ]:
#Compute beam diameter d(t)
import scipy.integrate


def diameter(intensity_array):
    x,y = intensity_array['x'],intensity_array['y']
    intensity = intensity_array.values
    integral_1 = scipy.integrate.trapezoid(scipy.integrate.trapezoid(intensity, x=y, axis=1), x=x, axis=0)**2
    integral_2 = scipy.integrate.trapezoid(scipy.integrate.trapezoid(intensity**2, x=y, axis=1), x=x, axis=0)

    return 2*np.sqrt((integral_1/integral_2)/np.pi)

In [ ]:
slices = [0.1,0.15,0.3,0.5,0.7,0.9,1.0]

plt.figure(figsize=(15, 10))
for path_direction in [
                       "RCP Sample Tight Beam Near Gap Slices 1/z_incidence"
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                field_time_center = sim_data.get_intensity("time_monitorFieldCenter")
                plt.plot(field_time_center['t']*1e12 ,diameter(field_time_center).squeeze()/a, label=f"L={slices[i]}")


plt.ylabel("d(t)(in units of a)")
plt.xlabel("Time[ps]")
plt.title("Diameter d(t) at center of the structure")
plt.legend()
plt.grid()
plt.show()
              




In [ ]:
plt.figure(figsize=(15, 10))
for path_direction in [
                       "RCP Sample Tight Beam Near Gap Slices 1/z_incidence"
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                field_time_out = sim_data.get_intensity("time_monitorFieldOut")
                plt.plot(field_time_out['t']*1e12 ,diameter(field_time_out).squeeze()/a, label=f"L={slices[i]}")
      





plt.ylabel("d(t)(in units of a)")
plt.xlabel("Time[ps]")
plt.title("Diameter d(t) at output of the structure")
plt.legend()
plt.grid()
plt.savefig(f'output/anderson/diameter in time.pdf', format='pdf')
plt.show()
              




In [ ]:
field_log = np.log10(field_time_out/np.max(field_time_out)+1e-3)


for i,time in enumerate(field_time_out['t']):
      if i%20==0:
          (field_log).isel(t=i).squeeze().plot.pcolormesh(cmap="YlOrRd")
          plt.show()
